In [2]:
import joblib
import pandas as pd

In [3]:
# scalar = joblib.load('robust_scalar.pkl')
# model = joblib.load('rf_model.pkl') 
model_with_pipeline = joblib.load('../models/rf_model_with_pipeline.pkl')

### Read the dataframe which has not been used in training 

In [4]:
df = pd.read_csv('../data/remaining_data.csv')


In [5]:
df

,focal_addr,focal_addr_type,blacklist,tot_txn,tot_val,min_val,max_val,sd2_val,unq_other_addr,days_diff_first_last,...,to_contract_txn,tot_val_to_contract,min_val_to_contract,max_val_to_contract,avg_val_to_contract,erc20_txn,sum_val_erc20,erc721_txn,sum_val_erc721,days_since_last_txn
0,0x000677bbab0fea85506f5f63806d331c9c2df1ac,wallet address,0.0,1.0,7.287733e+14,7.287733e+14,7.287733e+14,0.000000e+00,1.0,0.000000,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,178.0
1,0x002418dd34b76845aa272f34c13cdbeb5a23910c,wallet address,0.0,1.0,3.308973e+16,3.308973e+16,3.308973e+16,0.000000e+00,1.0,0.000000,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,107.0
2,0x00441c5adeba58b922109e299c42ae38d4ae36de,wallet address,0.0,38.0,2.097861e+18,0.000000e+00,5.456920e+17,1.216219e+34,19.0,53.878935,...,19.0,2.440000e+17,0.0,1.500000e+17,1.284211e+16,2.0,0.0,0.0,0.0,163.0
3,0x008365df318219673ade95072d94540270291ec2,wallet address,0.0,9.0,3.163486e+16,0.000000e+00,1.040000e+16,1.816578e+31,6.0,128.468461,...,5.0,1.040000e+16,0.0,1.040000e+16,2.080000e+15,0.0,0.0,0.0,0.0,36.0
4,0x008a9df11b66d7e14a41ffd78d1b4db21cb14943,wallet address,0.0,1.0,5.873961e+15,5.873961e+15,5.873961e+15,0.000000e+00,1.0,0.000000,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,0xfef63d0e41a7071fe0cd869e1f9faad0623d1cf6,wallet address,0.0,4.0,3.175245e+17,4.115089e+16,1.600000e+17,2.322585e+33,4.0,5.357083,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,27.0
1672,0xff5019880f5cd242f5283cbad27be0b573d12046,wallet address,0.0,1.0,2.687675e+16,2.687675e+16,2.687675e+16,0.000000e+00,1.0,0.000000,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,138.0
1673,0xff8f9e0a51441b2bbf3afb10ca98147ec51a78bb,wallet address,0.0,195.0,3.326411e+19,0.000000e+00,1.900000e+18,1.168418e+35,37.0,172.964410,...,162.0,2.024211e+19,0.0,1.800000e+18,1.249513e+17,1.0,0.0,0.0,0.0,3.0
1674,0xff9d98c441ef33f9280547c009a57a050763699d,wallet address,0.0,1.0,2.614362e+14,2.614362e+14,2.614362e+14,0.000000e+00,1.0,0.000000,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,36.0


### Create a function toprovide prediction score if an address is a good or bad

In [9]:
def predict(dataframe):
        selected_features = ['days_diff_first_last_sent',
                     'unq_to_addr',
                     'sent_txn',
                     'to_contract_txn',
                     'contract_creation',
                     'tot_txn',
                     'unq_other_addr',
                     'tot_val',
                     'tot_val_sent',
                     'max_val',
                     'avg_val_sent',
                     'tot_val_recv',
                     'recv_txn',
                     'max_val_sent',
                     'max_val_recv']
    
        model_with_pipeline = joblib.load('../models/rf_model_with_pipeline.pkl')

        predict_proba = model_with_pipeline.predict_proba(df[selected_features])

        # Combine the predicted probabilities with the original dataframe
        pred_df = pd.concat([df, pd.DataFrame(predict_proba, columns=model_with_pipeline.classes_)], axis=1)
            # Define a function to get the final prediction
        def get_final_prediction(row):
            max_prob = row.max()
            predicted_class = 'Bad Address' if row.idxmax()==1 else 'Good Address'
            return f"{max_prob*100} % probability of being {predicted_class}"

        # Apply the function to each row of the dataframe to get the final prediction
        pred_df['final_prediction'] = pred_df[model_with_pipeline.classes_].apply(get_final_prediction, axis=1)
        pred_df.drop(model_with_pipeline.classes_, axis=1)
        return pred_df

In [10]:
results  = predict(df)

In [11]:
results

,focal_addr,focal_addr_type,blacklist,tot_txn,tot_val,min_val,max_val,sd2_val,unq_other_addr,days_diff_first_last,...,max_val_to_contract,avg_val_to_contract,erc20_txn,sum_val_erc20,erc721_txn,sum_val_erc721,days_since_last_txn,0,1,final_prediction
0,0x000677bbab0fea85506f5f63806d331c9c2df1ac,wallet address,0.0,1.0,7.287733e+14,7.287733e+14,7.287733e+14,0.000000e+00,1.0,0.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,178.0,0.97,0.03,97.0 % probability of being Good Address
1,0x002418dd34b76845aa272f34c13cdbeb5a23910c,wallet address,0.0,1.0,3.308973e+16,3.308973e+16,3.308973e+16,0.000000e+00,1.0,0.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,107.0,0.96,0.04,96.0 % probability of being Good Address
2,0x00441c5adeba58b922109e299c42ae38d4ae36de,wallet address,0.0,38.0,2.097861e+18,0.000000e+00,5.456920e+17,1.216219e+34,19.0,53.878935,...,1.500000e+17,1.284211e+16,2.0,0.0,0.0,0.0,163.0,0.91,0.09,91.0 % probability of being Good Address
3,0x008365df318219673ade95072d94540270291ec2,wallet address,0.0,9.0,3.163486e+16,0.000000e+00,1.040000e+16,1.816578e+31,6.0,128.468461,...,1.040000e+16,2.080000e+15,0.0,0.0,0.0,0.0,36.0,0.95,0.05,95.0 % probability of being Good Address
4,0x008a9df11b66d7e14a41ffd78d1b4db21cb14943,wallet address,0.0,1.0,5.873961e+15,5.873961e+15,5.873961e+15,0.000000e+00,1.0,0.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,78.0,1.00,0.00,100.0 % probability of being Good Address
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,0xfef63d0e41a7071fe0cd869e1f9faad0623d1cf6,wallet address,0.0,4.0,3.175245e+17,4.115089e+16,1.600000e+17,2.322585e+33,4.0,5.357083,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,27.0,1.00,0.00,100.0 % probability of being Good Address
1672,0xff5019880f5cd242f5283cbad27be0b573d12046,wallet address,0.0,1.0,2.687675e+16,2.687675e+16,2.687675e+16,0.000000e+00,1.0,0.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,138.0,1.00,0.00,100.0 % probability of being Good Address
1673,0xff8f9e0a51441b2bbf3afb10ca98147ec51a78bb,wallet address,0.0,195.0,3.326411e+19,0.000000e+00,1.900000e+18,1.168418e+35,37.0,172.964410,...,1.800000e+18,1.249513e+17,1.0,0.0,0.0,0.0,3.0,0.81,0.19,81.0 % probability of being Good Address
1674,0xff9d98c441ef33f9280547c009a57a050763699d,wallet address,0.0,1.0,2.614362e+14,2.614362e+14,2.614362e+14,0.000000e+00,1.0,0.000000,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,36.0,1.00,0.00,100.0 % probability of being Good Address
